Torch Model

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
data = pd.read_csv('../Data/clean_data.csv')
data = data.to_numpy()
data = np.delete(data, 8, 1)
data = np.delete(data, 8, 1)
data = data.astype('float64')
np.random.shuffle(data)
class_label = data[:, -1] # for last column
class_label = class_label.astype('int64') 
data = data[:, :-1] # for all but last column
print(data.shape)
print(class_label.shape)

(51598, 9)
(51598,)


In [3]:
x_train = data[0:49000, :]
y_train = class_label[0:49000] 
x_val = data[49000:50000, :]
y_val = class_label[49000:50000] 
x_test = data[50000:, :]
y_test = class_label[50000:] 

In [4]:
print("Training Set Data  Shape: ", x_train.shape)
print("Training Set Label Shape: ", y_train.shape)
print("Validation Set Data  Shape: ", x_val.shape)
print("Validation Set Label Shape: ", y_val.shape)
print("Test Set Data  Shape: ", x_test.shape)
print("Test Set Label Shape: ", y_test.shape)

Training Set Data  Shape:  (49000, 9)
Training Set Label Shape:  (49000,)
Validation Set Data  Shape:  (1000, 9)
Validation Set Label Shape:  (1000,)
Test Set Data  Shape:  (1598, 9)
Test Set Label Shape:  (1598,)


In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
print(torch.cuda.get_device_name(0))
dtype = torch.float32
device = torch.device('cuda')

GeForce RTX 2080 Super with Max-Q Design


In [6]:
x_train = torch.from_numpy(x_train)
y_train = torch.from_numpy(y_train)
x_val = torch.from_numpy(x_val)
y_val = torch.from_numpy(y_val)
x_test = torch.from_numpy(x_test)
y_test = torch.from_numpy(y_test)

In [7]:
print("Training Set Data  Shape: ", x_train.size())
print("Training Set Label Shape: ", y_train.size())
print("Validation Set Data  Shape: ", x_val.size())
print("Validation Set Label Shape: ", y_val.size())
print("Test Set Data  Shape: ", x_test.size())
print("Test Set Label Shape: ", y_test.size())

Training Set Data  Shape:  torch.Size([49000, 9])
Training Set Label Shape:  torch.Size([49000])
Validation Set Data  Shape:  torch.Size([1000, 9])
Validation Set Label Shape:  torch.Size([1000])
Test Set Data  Shape:  torch.Size([1598, 9])
Test Set Label Shape:  torch.Size([1598])


Barebone API

In [8]:
def random_weight(shape):
    """
    Create random Tensors for weights; setting requires_grad=True means that we
    want to compute gradients for these Tensors during the backward pass.
    We use Kaiming normalization: sqrt(2 / fan_in)
    """
    if len(shape) == 2:  # FC weight
        fan_in = shape[0]
    else:
        fan_in = np.prod(shape[1:]) # conv weight [out_channel, in_channel, kH, kW]
    # randn is standard normal distribution generator. 
    w = torch.randn(shape, device=device, dtype=dtype) * np.sqrt(2. / fan_in)
    w.requires_grad = True
    return w

def zero_weight(shape):
    return torch.zeros(shape, device=device, dtype=dtype, requires_grad=True)

In [54]:
def check_accuracy(model_fn, params):
    print('Checking accuracy on the validation set')
    num_correct, num_samples = 0, 0
    with torch.no_grad():
        x_val_gpu = x_train.to(device=device, dtype=dtype)  # move to device, e.g. GPU
        y_val_gpu = y_train.to(device=device, dtype=torch.int64)
        scores = model_fn(x_val_gpu, params)
        _, preds = scores.max(1)
        num_correct += (preds == y_val_gpu).sum()
        num_samples += preds.size(0)
    acc = float(num_correct) / num_samples
    print('Got %d / %d correct (%.2f%%)' % (num_correct, num_samples, 100 * acc))

In [48]:
def train(model_fn, params, learning_rate):
    # Move the data to the proper device (GPU or CPU)
    x_train_gpu = x_train.to(device=device, dtype=dtype)
    y_train_gpu = y_train.to(device=device, dtype=torch.long)
    
    for i in range(10000):
        # Forward pass: compute scores and loss
        scores = model_fn(x_train_gpu, params)
        loss = F.cross_entropy(scores, y_train_gpu)

        # Backward pass: PyTorch figures out which Tensors in the computational
        # graph has requires_grad=True and uses backpropagation to compute the
        # gradient of the loss with respect to these Tensors, and stores the
        # gradients in the .grad attribute of each Tensor.
        loss.backward()

        # Update parameters. We don't want to backpropagate through the
        # parameter updates, so we scope the updates under a torch.no_grad()
        # context manager to prevent a computational graph from being built.
        with torch.no_grad():
            for w in params:
                w -= learning_rate * w.grad

                # Manually zero the gradients after running the backward pass
                w.grad.zero_()

        print('loss = %.4f' % (loss.item()))
        check_accuracy(model_fn, params)
        print()

In [50]:
import torch.nn.functional as F  # useful stateless functions

def two_layer_fc(x, params):   
    w1, b1, w2, b2, w3, b3, w4, b4, w5, b5, w6, b6, w7, b7 = params
    x = F.relu(x.mm(w1) + b1)
    x = F.relu(x.mm(w2) + b2)
    x = F.relu(x.mm(w3) + b3)
    x = F.relu(x.mm(w4) + b4)
    x = F.relu(x.mm(w5) + b5)
    x = F.relu(x.mm(w6) + b6)
    x = x.mm(w7) + b7
    return x

In [58]:
hidden_layer_size_1 = 8
hidden_layer_size_2 = 7
hidden_layer_size_3 = 6
hidden_layer_size_4 = 5
hidden_layer_size_5 = 4
hidden_layer_size_6 = 3
learning_rate = 1e-3

w1 = random_weight((9, hidden_layer_size_1))
b1 = zero_weight(hidden_layer_size_1)

w2 = random_weight((hidden_layer_size_1, hidden_layer_size_2))
b2 = zero_weight(hidden_layer_size_2)

w3 = random_weight((hidden_layer_size_2, hidden_layer_size_3))
b3 = zero_weight(hidden_layer_size_3)

w4 = random_weight((hidden_layer_size_3, hidden_layer_size_4))
b4 = zero_weight(hidden_layer_size_4)

w5 = random_weight((hidden_layer_size_4, hidden_layer_size_5))
b5 = zero_weight(hidden_layer_size_5)

w6 = random_weight((hidden_layer_size_5, hidden_layer_size_6))
b6 = zero_weight(hidden_layer_size_6)

w7 = random_weight((hidden_layer_size_6, 2))
b7 = zero_weight(2)

train(two_layer_fc, [w1, b1, w2, b2, w3, b3, w4, b4, w5, b5, w6, b6, w7, b7], learning_rate)

loss = 2.9772
Checking accuracy on the validation set
Got 25166 / 49000 correct (51.36%)

loss = 2.5802
Checking accuracy on the validation set
Got 25181 / 49000 correct (51.39%)

loss = 1.8156
Checking accuracy on the validation set
Got 25445 / 49000 correct (51.93%)

loss = 0.6937
Checking accuracy on the validation set
Got 25370 / 49000 correct (51.78%)

loss = 0.6935
Checking accuracy on the validation set
Got 25321 / 49000 correct (51.68%)

loss = 0.6934
Checking accuracy on the validation set
Got 25300 / 49000 correct (51.63%)

loss = 0.6934
Checking accuracy on the validation set
Got 25284 / 49000 correct (51.60%)

loss = 0.6933
Checking accuracy on the validation set
Got 25258 / 49000 correct (51.55%)

loss = 0.6933
Checking accuracy on the validation set
Got 25221 / 49000 correct (51.47%)

loss = 0.6933
Checking accuracy on the validation set
Got 25217 / 49000 correct (51.46%)

loss = 0.6932
Checking accuracy on the validation set
Got 25210 / 49000 correct (51.45%)

loss = 0.6

loss = 0.6924
Checking accuracy on the validation set
Got 25502 / 49000 correct (52.04%)

loss = 0.6924
Checking accuracy on the validation set
Got 25500 / 49000 correct (52.04%)

loss = 0.6924
Checking accuracy on the validation set
Got 25501 / 49000 correct (52.04%)

loss = 0.6924
Checking accuracy on the validation set
Got 25501 / 49000 correct (52.04%)

loss = 0.6924
Checking accuracy on the validation set
Got 25501 / 49000 correct (52.04%)

loss = 0.6924
Checking accuracy on the validation set
Got 25497 / 49000 correct (52.03%)

loss = 0.6924
Checking accuracy on the validation set
Got 25496 / 49000 correct (52.03%)

loss = 0.6924
Checking accuracy on the validation set
Got 25501 / 49000 correct (52.04%)

loss = 0.6924
Checking accuracy on the validation set
Got 25525 / 49000 correct (52.09%)

loss = 0.6923
Checking accuracy on the validation set
Got 25561 / 49000 correct (52.17%)

loss = 0.6923
Checking accuracy on the validation set
Got 25575 / 49000 correct (52.19%)

loss = 0.6

Got 26164 / 49000 correct (53.40%)

loss = 0.6908
Checking accuracy on the validation set
Got 26160 / 49000 correct (53.39%)

loss = 0.6908
Checking accuracy on the validation set
Got 26164 / 49000 correct (53.40%)

loss = 0.6907
Checking accuracy on the validation set
Got 26166 / 49000 correct (53.40%)

loss = 0.6907
Checking accuracy on the validation set
Got 26168 / 49000 correct (53.40%)

loss = 0.6907
Checking accuracy on the validation set
Got 26170 / 49000 correct (53.41%)

loss = 0.6907
Checking accuracy on the validation set
Got 26171 / 49000 correct (53.41%)

loss = 0.6907
Checking accuracy on the validation set
Got 26170 / 49000 correct (53.41%)

loss = 0.6907
Checking accuracy on the validation set
Got 26169 / 49000 correct (53.41%)

loss = 0.6907
Checking accuracy on the validation set
Got 26171 / 49000 correct (53.41%)

loss = 0.6907
Checking accuracy on the validation set
Got 26182 / 49000 correct (53.43%)

loss = 0.6906
Checking accuracy on the validation set
Got 26183 

loss = 0.6891
Checking accuracy on the validation set
Got 26740 / 49000 correct (54.57%)

loss = 0.6891
Checking accuracy on the validation set
Got 26736 / 49000 correct (54.56%)

loss = 0.6890
Checking accuracy on the validation set
Got 26741 / 49000 correct (54.57%)

loss = 0.6890
Checking accuracy on the validation set
Got 26746 / 49000 correct (54.58%)

loss = 0.6890
Checking accuracy on the validation set
Got 26750 / 49000 correct (54.59%)

loss = 0.6890
Checking accuracy on the validation set
Got 26752 / 49000 correct (54.60%)

loss = 0.6890
Checking accuracy on the validation set
Got 26760 / 49000 correct (54.61%)

loss = 0.6890
Checking accuracy on the validation set
Got 26763 / 49000 correct (54.62%)

loss = 0.6889
Checking accuracy on the validation set
Got 26771 / 49000 correct (54.63%)

loss = 0.6889
Checking accuracy on the validation set
Got 26791 / 49000 correct (54.68%)

loss = 0.6889
Checking accuracy on the validation set
Got 26794 / 49000 correct (54.68%)

loss = 0.6

loss = 0.6868
Checking accuracy on the validation set
Got 27317 / 49000 correct (55.75%)

loss = 0.6868
Checking accuracy on the validation set
Got 27313 / 49000 correct (55.74%)

loss = 0.6868
Checking accuracy on the validation set
Got 27319 / 49000 correct (55.75%)

loss = 0.6868
Checking accuracy on the validation set
Got 27315 / 49000 correct (55.74%)

loss = 0.6867
Checking accuracy on the validation set
Got 27321 / 49000 correct (55.76%)

loss = 0.6867
Checking accuracy on the validation set
Got 27330 / 49000 correct (55.78%)

loss = 0.6867
Checking accuracy on the validation set
Got 27346 / 49000 correct (55.81%)

loss = 0.6867
Checking accuracy on the validation set
Got 27368 / 49000 correct (55.85%)

loss = 0.6866
Checking accuracy on the validation set
Got 27377 / 49000 correct (55.87%)

loss = 0.6866
Checking accuracy on the validation set
Got 27403 / 49000 correct (55.92%)

loss = 0.6866
Checking accuracy on the validation set
Got 27427 / 49000 correct (55.97%)

loss = 0.6

...

loss = 0.6678
Checking accuracy on the validation set
Got 29340 / 49000 correct (59.88%)

loss = 0.6678
Checking accuracy on the validation set
Got 29340 / 49000 correct (59.88%)

loss = 0.6678
Checking accuracy on the validation set
Got 29340 / 49000 correct (59.88%)

loss = 0.6678
Checking accuracy on the validation set
Got 29340 / 49000 correct (59.88%)

loss = 0.6678
Checking accuracy on the validation set
Got 29340 / 49000 correct (59.88%)

loss = 0.6678
Checking accuracy on the validation set
Got 29340 / 49000 correct (59.88%)

loss = 0.6678
Checking accuracy on the validation set
Got 29340 / 49000 correct (59.88%)

loss = 0.6678
Checking accuracy on the validation set
Got 29340 / 49000 correct (59.88%)

loss = 0.6678
Checking accuracy on the validation set
Got 29340 / 49000 correct (59.88%)

loss = 0.6678
Checking accuracy on the validation set
Got 29340 / 49000 correct (59.88%)

loss = 0.6678
Checking accuracy on the validation set
Got 29340 / 49000 correct (59.88%)

loss = 0.6

loss = 0.6678
Checking accuracy on the validation set
Got 29342 / 49000 correct (59.88%)

loss = 0.6678
Checking accuracy on the validation set
Got 29345 / 49000 correct (59.89%)

loss = 0.6678
Checking accuracy on the validation set
Got 29342 / 49000 correct (59.88%)

loss = 0.6678
Checking accuracy on the validation set
Got 29346 / 49000 correct (59.89%)

loss = 0.6678
Checking accuracy on the validation set
Got 29342 / 49000 correct (59.88%)

loss = 0.6678
Checking accuracy on the validation set
Got 29344 / 49000 correct (59.89%)

loss = 0.6678
Checking accuracy on the validation set
Got 29341 / 49000 correct (59.88%)

loss = 0.6678
Checking accuracy on the validation set
Got 29345 / 49000 correct (59.89%)

loss = 0.6678
Checking accuracy on the validation set
Got 29340 / 49000 correct (59.88%)

loss = 0.6678
Checking accuracy on the validation set
Got 29345 / 49000 correct (59.89%)

loss = 0.6678
Checking accuracy on the validation set
Got 29341 / 49000 correct (59.88%)

loss = 0.6

Got 29344 / 49000 correct (59.89%)

loss = 0.6678
Checking accuracy on the validation set
Got 29345 / 49000 correct (59.89%)

loss = 0.6678
Checking accuracy on the validation set
Got 29344 / 49000 correct (59.89%)

loss = 0.6678
Checking accuracy on the validation set
Got 29346 / 49000 correct (59.89%)

loss = 0.6678
Checking accuracy on the validation set
Got 29342 / 49000 correct (59.88%)

loss = 0.6678
Checking accuracy on the validation set
Got 29345 / 49000 correct (59.89%)

loss = 0.6678
Checking accuracy on the validation set
Got 29342 / 49000 correct (59.88%)

loss = 0.6678
Checking accuracy on the validation set
Got 29345 / 49000 correct (59.89%)

loss = 0.6678
Checking accuracy on the validation set
Got 29341 / 49000 correct (59.88%)

loss = 0.6678
Checking accuracy on the validation set
Got 29345 / 49000 correct (59.89%)

loss = 0.6678
Checking accuracy on the validation set
Got 29341 / 49000 correct (59.88%)

loss = 0.6678
Checking accuracy on the validation set
Got 29345 

loss = 0.6677
Checking accuracy on the validation set
Got 29347 / 49000 correct (59.89%)

loss = 0.6678
Checking accuracy on the validation set
Got 29348 / 49000 correct (59.89%)

loss = 0.6678
Checking accuracy on the validation set
Got 29345 / 49000 correct (59.89%)

loss = 0.6678
Checking accuracy on the validation set
Got 29349 / 49000 correct (59.90%)

loss = 0.6678
Checking accuracy on the validation set
Got 29345 / 49000 correct (59.89%)

loss = 0.6678
Checking accuracy on the validation set
Got 29350 / 49000 correct (59.90%)

loss = 0.6678
Checking accuracy on the validation set
Got 29345 / 49000 correct (59.89%)

loss = 0.6678
Checking accuracy on the validation set
Got 29350 / 49000 correct (59.90%)

loss = 0.6678
Checking accuracy on the validation set
Got 29345 / 49000 correct (59.89%)

loss = 0.6678
Checking accuracy on the validation set
Got 29349 / 49000 correct (59.90%)

loss = 0.6678
Checking accuracy on the validation set
Got 29344 / 49000 correct (59.89%)

loss = 0.6

loss = 0.6677
Checking accuracy on the validation set
Got 29346 / 49000 correct (59.89%)

loss = 0.6677
Checking accuracy on the validation set
Got 29348 / 49000 correct (59.89%)

loss = 0.6677
Checking accuracy on the validation set
Got 29345 / 49000 correct (59.89%)

loss = 0.6677
Checking accuracy on the validation set
Got 29348 / 49000 correct (59.89%)

loss = 0.6677
Checking accuracy on the validation set
Got 29347 / 49000 correct (59.89%)

loss = 0.6677
Checking accuracy on the validation set
Got 29348 / 49000 correct (59.89%)

loss = 0.6677
Checking accuracy on the validation set
Got 29345 / 49000 correct (59.89%)

loss = 0.6677
Checking accuracy on the validation set
Got 29349 / 49000 correct (59.90%)

loss = 0.6677
Checking accuracy on the validation set
Got 29348 / 49000 correct (59.89%)

loss = 0.6677
Checking accuracy on the validation set
Got 29349 / 49000 correct (59.90%)

loss = 0.6677
Checking accuracy on the validation set
Got 29348 / 49000 correct (59.89%)

loss = 0.6

nn.Module API

In [61]:
def check_accuracy_module(model):
    print('Checking accuracy on training set')  
    num_correct = 0
    num_samples = 0
    model.eval()  # set model to evaluation mode
    with torch.no_grad():
        x_train_gpu = x_train.to(device=device, dtype=dtype)  # move to device, e.g. GPU
        y_train_gpu = y_train.to(device=device, dtype=torch.long)
        scores = model(x_train_gpu)
        _, preds = scores.max(1)
        num_correct += (preds == y_train_gpu).sum()
        num_samples += preds.size(0)
    acc = float(num_correct) / num_samples
    print('Got %d / %d correct (%.2f)' % (num_correct, num_samples, 100 * acc))

In [62]:
def train_module(model, optimizer, epochs=1):
    model = model.to(device=device)  # move the model parameters to CPU/GPU
    for e in range(epochs):
        model.train()  # put model to training mode
        x_train_gpu = x_train.to(device=device, dtype=dtype)  # move to device, e.g. GPU
        y_train_gpu = y_train.to(device=device, dtype=torch.long)

        scores = model(x_train_gpu)
        loss = F.cross_entropy(scores, y_train_gpu)

        # Zero out all of the gradients for the variables which the optimizer
        # will update.
        optimizer.zero_grad()

        # This is the backwards pass: compute the gradient of the loss with
        # respect to each  parameter of the model.
        loss.backward()

        # Actually update the parameters of the model using the gradients
        # computed by the backwards pass.
        optimizer.step()

        if e % 100 == 0:
            print('Iteration %d, loss = %.4f' % (e, loss.item()))
            check_accuracy_module(model)
            print()

In [59]:
class TwoLayerFC(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super().__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        #nn.init.kaiming_normal_(self.fc1.weight)
        self.fc2 = nn.Linear(hidden_size, num_classes)
        #nn.init.kaiming_normal_(self.fc2.weight)
    
    def forward(self, x):
        scores = self.fc2(F.relu(self.fc1(x)))
        return scores

In [72]:
hidden_layer_size = 8
learning_rate = 1e-3
model_fc = TwoLayerFC(9, hidden_layer_size, 2)
optimizer = optim.SGD(model_fc.parameters(), lr=learning_rate)
optimizer_1 = optim.SGD(model_fc.parameters(), lr=learning_rate/2)

train_module(model_fc, optimizer, 30000)
train_module(model_fc, optimizer_1, 10000)

Iteration 0, loss = 1.4093
Checking accuracy on training set
Got 23884 / 49000 correct (48.74)

Iteration 100, loss = 0.7282
Checking accuracy on training set
Got 27858 / 49000 correct (56.85)

Iteration 200, loss = 0.7054
Checking accuracy on training set
Got 27959 / 49000 correct (57.06)

Iteration 300, loss = 0.6879
Checking accuracy on training set
Got 28069 / 49000 correct (57.28)

Iteration 400, loss = 0.6816
Checking accuracy on training set
Got 28420 / 49000 correct (58.00)

Iteration 500, loss = 0.6784
Checking accuracy on training set
Got 28584 / 49000 correct (58.33)

Iteration 600, loss = 0.6760
Checking accuracy on training set
Got 28696 / 49000 correct (58.56)

Iteration 700, loss = 0.6743
Checking accuracy on training set
Got 28804 / 49000 correct (58.78)

Iteration 800, loss = 0.6733
Checking accuracy on training set
Got 28865 / 49000 correct (58.91)

Iteration 900, loss = 0.6725
Checking accuracy on training set
Got 28932 / 49000 correct (59.04)

Iteration 1000, loss =

Iteration 8300, loss = 0.6674
Checking accuracy on training set
Got 29397 / 49000 correct (59.99)

Iteration 8400, loss = 0.6674
Checking accuracy on training set
Got 29404 / 49000 correct (60.01)

Iteration 8500, loss = 0.6673
Checking accuracy on training set
Got 29405 / 49000 correct (60.01)

Iteration 8600, loss = 0.6673
Checking accuracy on training set
Got 29396 / 49000 correct (59.99)

Iteration 8700, loss = 0.6673
Checking accuracy on training set
Got 29398 / 49000 correct (60.00)

Iteration 8800, loss = 0.6673
Checking accuracy on training set
Got 29400 / 49000 correct (60.00)

Iteration 8900, loss = 0.6673
Checking accuracy on training set
Got 29401 / 49000 correct (60.00)

Iteration 9000, loss = 0.6673
Checking accuracy on training set
Got 29398 / 49000 correct (60.00)

Iteration 9100, loss = 0.6673
Checking accuracy on training set
Got 29395 / 49000 correct (59.99)

Iteration 9200, loss = 0.6673
Checking accuracy on training set
Got 29397 / 49000 correct (59.99)

Iteration 

Iteration 16600, loss = 0.6667
Checking accuracy on training set
Got 29438 / 49000 correct (60.08)

Iteration 16700, loss = 0.6667
Checking accuracy on training set
Got 29439 / 49000 correct (60.08)

Iteration 16800, loss = 0.6667
Checking accuracy on training set
Got 29438 / 49000 correct (60.08)

Iteration 16900, loss = 0.6667
Checking accuracy on training set
Got 29437 / 49000 correct (60.08)

Iteration 17000, loss = 0.6667
Checking accuracy on training set
Got 29439 / 49000 correct (60.08)

Iteration 17100, loss = 0.6667
Checking accuracy on training set
Got 29440 / 49000 correct (60.08)

Iteration 17200, loss = 0.6667
Checking accuracy on training set
Got 29439 / 49000 correct (60.08)

Iteration 17300, loss = 0.6667
Checking accuracy on training set
Got 29438 / 49000 correct (60.08)

Iteration 17400, loss = 0.6667
Checking accuracy on training set
Got 29438 / 49000 correct (60.08)

Iteration 17500, loss = 0.6666
Checking accuracy on training set
Got 29439 / 49000 correct (60.08)



Iteration 24800, loss = 0.6663
Checking accuracy on training set
Got 29442 / 49000 correct (60.09)

Iteration 24900, loss = 0.6663
Checking accuracy on training set
Got 29443 / 49000 correct (60.09)

Iteration 25000, loss = 0.6663
Checking accuracy on training set
Got 29443 / 49000 correct (60.09)

Iteration 25100, loss = 0.6663
Checking accuracy on training set
Got 29443 / 49000 correct (60.09)

Iteration 25200, loss = 0.6663
Checking accuracy on training set
Got 29443 / 49000 correct (60.09)

Iteration 25300, loss = 0.6663
Checking accuracy on training set
Got 29443 / 49000 correct (60.09)

Iteration 25400, loss = 0.6663
Checking accuracy on training set
Got 29443 / 49000 correct (60.09)

Iteration 25500, loss = 0.6663
Checking accuracy on training set
Got 29443 / 49000 correct (60.09)

Iteration 25600, loss = 0.6663
Checking accuracy on training set
Got 29442 / 49000 correct (60.09)

Iteration 25700, loss = 0.6663
Checking accuracy on training set
Got 29441 / 49000 correct (60.08)



Iteration 3100, loss = 0.6661
Checking accuracy on training set
Got 29452 / 49000 correct (60.11)

Iteration 3200, loss = 0.6661
Checking accuracy on training set
Got 29452 / 49000 correct (60.11)

Iteration 3300, loss = 0.6661
Checking accuracy on training set
Got 29453 / 49000 correct (60.11)

Iteration 3400, loss = 0.6661
Checking accuracy on training set
Got 29453 / 49000 correct (60.11)

Iteration 3500, loss = 0.6661
Checking accuracy on training set
Got 29452 / 49000 correct (60.11)

Iteration 3600, loss = 0.6661
Checking accuracy on training set
Got 29452 / 49000 correct (60.11)

Iteration 3700, loss = 0.6661
Checking accuracy on training set
Got 29452 / 49000 correct (60.11)

Iteration 3800, loss = 0.6661
Checking accuracy on training set
Got 29452 / 49000 correct (60.11)

Iteration 3900, loss = 0.6661
Checking accuracy on training set
Got 29452 / 49000 correct (60.11)

Iteration 4000, loss = 0.6661
Checking accuracy on training set
Got 29453 / 49000 correct (60.11)

Iteration 

In [73]:
b = torch.tensor([[0, 1, 2, 3, 4, 5, 6, 7, 8], [9, 10, 11, 12, 13, 14, 15, 16, 17]])
torch.reshape(b, (2,1,3,3))

tensor([[[[ 0,  1,  2],
          [ 3,  4,  5],
          [ 6,  7,  8]]],


        [[[ 9, 10, 11],
          [12, 13, 14],
          [15, 16, 17]]]])

In [95]:
x_train_reshape = torch.reshape(x_train, (49000, 1, 3, 3))
x_val_reshape = torch.reshape(x_val, (1000, 1, 3, 3))
x_test_reshape = torch.reshape(x_test, (1598, 1, 3, 3))

In [75]:
def flatten(x):
    N = x.shape[0] # read in N, C, H, W
    return x.view(N, -1)  # "flatten" the C * H * W values into a single vector per image

In [76]:
class ThreeLayerConvNet(nn.Module):
    def __init__(self, in_channel, channel_1, channel_2, num_classes):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channel, channel_1, 3, padding=1)
        self.conv2 = nn.Conv2d(channel_1, channel_2, 3, padding=1)
        self.fc1 = nn.Linear(channel_2 * 3 * 3, num_classes)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = flatten(x)
        scores = self.fc1(x)
        return scores

In [80]:
def train_module_conv(model, optimizer, epochs=1):
    model = model.to(device=device)  # move the model parameters to CPU/GPU
    for e in range(epochs):
        model.train()  # put model to training mode
        x_train_gpu = x_train_reshape.to(device=device, dtype=dtype)  # move to device, e.g. GPU
        y_train_gpu = y_train.to(device=device, dtype=torch.long)

        scores = model(x_train_gpu)
        loss = F.cross_entropy(scores, y_train_gpu)

        # Zero out all of the gradients for the variables which the optimizer
        # will update.
        optimizer.zero_grad()

        # This is the backwards pass: compute the gradient of the loss with
        # respect to each  parameter of the model.
        loss.backward()

        # Actually update the parameters of the model using the gradients
        # computed by the backwards pass.
        optimizer.step()

        if e % 100 == 0:
            print('Iteration %d, loss = %.4f' % (e, loss.item()))
            check_accuracy_module_conv(model)
            print()

In [81]:
def check_accuracy_module_conv(model):
    print('Checking accuracy on training set')  
    num_correct = 0
    num_samples = 0
    model.eval()  # set model to evaluation mode
    with torch.no_grad():
        x_train_gpu = x_train_reshape.to(device=device, dtype=dtype)  # move to device, e.g. GPU
        y_train_gpu = y_train.to(device=device, dtype=torch.long)
        scores = model(x_train_gpu)
        _, preds = scores.max(1)
        num_correct += (preds == y_train_gpu).sum()
        num_samples += preds.size(0)
    acc = float(num_correct) / num_samples
    print('Got %d / %d correct (%.2f)' % (num_correct, num_samples, 100 * acc))

In [82]:
learning_rate = 3e-3
channel_1 = 32
channel_2 = 16

model_conv = ThreeLayerConvNet(1, channel_1, channel_2, 2)
optimizer = optim.SGD(model_conv.parameters(), lr=learning_rate)

train_module_conv(model_conv, optimizer, 10000)

Iteration 0, loss = 2.3452
Checking accuracy on training set
Got 25112 / 49000 correct (51.25)

Iteration 100, loss = 0.6667
Checking accuracy on training set
Got 29494 / 49000 correct (60.19)

Iteration 200, loss = 0.6667
Checking accuracy on training set
Got 29491 / 49000 correct (60.19)

Iteration 300, loss = 0.6663
Checking accuracy on training set
Got 29504 / 49000 correct (60.21)

Iteration 400, loss = 0.6661
Checking accuracy on training set
Got 29517 / 49000 correct (60.24)

Iteration 500, loss = 0.6657
Checking accuracy on training set
Got 29489 / 49000 correct (60.18)

Iteration 600, loss = 0.6657
Checking accuracy on training set
Got 29508 / 49000 correct (60.22)

Iteration 700, loss = 0.6657
Checking accuracy on training set
Got 29511 / 49000 correct (60.23)

Iteration 800, loss = 0.6655
Checking accuracy on training set
Got 29488 / 49000 correct (60.18)

Iteration 900, loss = 0.6655
Checking accuracy on training set
Got 29496 / 49000 correct (60.20)

Iteration 1000, loss =

Iteration 8300, loss = 0.6646
Checking accuracy on training set
Got 29526 / 49000 correct (60.26)

Iteration 8400, loss = 0.6646
Checking accuracy on training set
Got 29527 / 49000 correct (60.26)

Iteration 8500, loss = 0.6646
Checking accuracy on training set
Got 29530 / 49000 correct (60.27)

Iteration 8600, loss = 0.6646
Checking accuracy on training set
Got 29533 / 49000 correct (60.27)

Iteration 8700, loss = 0.6646
Checking accuracy on training set
Got 29527 / 49000 correct (60.26)

Iteration 8800, loss = 0.6646
Checking accuracy on training set
Got 29527 / 49000 correct (60.26)

Iteration 8900, loss = 0.6646
Checking accuracy on training set
Got 29528 / 49000 correct (60.26)

Iteration 9000, loss = 0.6646
Checking accuracy on training set
Got 29524 / 49000 correct (60.25)

Iteration 9100, loss = 0.6646
Checking accuracy on training set
Got 29534 / 49000 correct (60.27)

Iteration 9200, loss = 0.6646
Checking accuracy on training set
Got 29531 / 49000 correct (60.27)

Iteration 

In [94]:
print("training set")
tp = 0
fp = 0
fn = 0
num_correct = 0
num_samples = 0
model_conv.eval()  # set model to evaluation mode
with torch.no_grad():
    x_train_gpu = x_train_reshape.to(device=device, dtype=dtype)  # move to device, e.g. GPU
    y_train_gpu = y_train.to(device=device, dtype=torch.long)
    scores = model_conv(x_train_gpu)
    scores, preds = scores.max(1)
    preds_np = preds.cpu().numpy()
    y_train_gpu_np = y_train_gpu.cpu().numpy()
    num_correct += (preds == y_train_gpu).sum()
    num_samples += preds.size(0)
acc = float(num_correct) / num_samples
print('Got %d / %d correct (%.2f)' % (num_correct, num_samples, 100 * acc))
for i in range(preds_np.shape[0]):
    if y_train_gpu_np[i] == 1:
        if preds_np[i] == 1:
            tp += 1
        else:
            fn += 1
    if y_train_gpu_np[i] == 0:
        if preds_np[i] == 1:
            fp += 1
            
print("tp: ", tp)
print("fp: ", fp)
print("fn: ", fn)
print("precision: ", tp/(tp+fp))
print("recall: ", tp/(tp+fn))

training set
Got 29480 / 49000 correct (60.16)
tp:  14924
fp:  9332
fn:  10188
precision:  0.6152704485488126
recall:  0.5942975469894871


In [96]:
print("validation set")
tp = 0
fp = 0
fn = 0
num_correct = 0
num_samples = 0
model_conv.eval()  # set model to evaluation mode
with torch.no_grad():
    x_train_gpu = x_val_reshape.to(device=device, dtype=dtype)  # move to device, e.g. GPU
    y_train_gpu = y_val.to(device=device, dtype=torch.long)
    scores = model_conv(x_train_gpu)
    scores, preds = scores.max(1)
    preds_np = preds.cpu().numpy()
    y_train_gpu_np = y_train_gpu.cpu().numpy()
    num_correct += (preds == y_train_gpu).sum()
    num_samples += preds.size(0)
acc = float(num_correct) / num_samples
print('Got %d / %d correct (%.2f)' % (num_correct, num_samples, 100 * acc))
for i in range(preds_np.shape[0]):
    if y_train_gpu_np[i] == 1:
        if preds_np[i] == 1:
            tp += 1
        else:
            fn += 1
    if y_train_gpu_np[i] == 0:
        if preds_np[i] == 1:
            fp += 1
            
print("tp: ", tp)
print("fp: ", fp)
print("fn: ", fn)
print("precision: ", tp/(tp+fp))
print("recall: ", tp/(tp+fn))

validation set
Got 599 / 1000 correct (59.90)
tp:  298
fp:  183
fn:  218
precision:  0.6195426195426196
recall:  0.5775193798449613


In [97]:
print("testing set")
tp = 0
fp = 0
fn = 0
num_correct = 0
num_samples = 0
model_conv.eval()  # set model to evaluation mode
with torch.no_grad():
    x_train_gpu = x_test_reshape.to(device=device, dtype=dtype)  # move to device, e.g. GPU
    y_train_gpu = y_test.to(device=device, dtype=torch.long)
    scores = model_conv(x_train_gpu)
    scores, preds = scores.max(1)
    preds_np = preds.cpu().numpy()
    y_train_gpu_np = y_train_gpu.cpu().numpy()
    num_correct += (preds == y_train_gpu).sum()
    num_samples += preds.size(0)
acc = float(num_correct) / num_samples
print('Got %d / %d correct (%.2f)' % (num_correct, num_samples, 100 * acc))
for i in range(preds_np.shape[0]):
    if y_train_gpu_np[i] == 1:
        if preds_np[i] == 1:
            tp += 1
        else:
            fn += 1
    if y_train_gpu_np[i] == 0:
        if preds_np[i] == 1:
            fp += 1
            
print("tp: ", tp)
print("fp: ", fp)
print("fn: ", fn)
print("precision: ", tp/(tp+fp))
print("recall: ", tp/(tp+fn))

testing set
Got 968 / 1598 correct (60.58)
tp:  499
fp:  329
fn:  301
precision:  0.6026570048309179
recall:  0.62375
